In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/sao-paulo-real-estate-sale-rent-april-2019/sao-paulo-properties-april-2019.csv')

Knowing more about our dataset

In [ ]:
data.head()

Checking null values

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
data.dtypes

Types of our data looks like ok

In [ ]:
# Checking unique values for negotition types
data['Negotiation Type'].unique(), data['Property Type'].unique()


 In this code we can conclude that in this dataset we only have appartaments for sale or rent in SP

In [ ]:
# We have a good proportions of values between our dataset, this could be nice for us to 
#maybe predict which appartment is on rent or sale,
#or maybe create a different model for prices according to the negotiation type
data['Negotiation Type'].value_counts()

In [ ]:
# to split data we have to distinguish wich columns we want to split, in our case im splitting District and city
data[['District','City']] = data['District'].str.split('/', expand = True)

In [ ]:
data.head()

In [ ]:
data.describe()

Strange values coming up the mean of price is 2.877, std aswell, weird. Let's found out

In [ ]:
## So we can drop the last column City from our dataset
data = data.drop(columns = 'City', axis = 1)
data.head()

## Exploratory Data Analysis (EDA)


First question, how many apartments we have for sale anf for rent ?



In [ ]:
plt.figure(figsize = (12,8))
ax = sns.countplot(x = 'Negotiation Type', data = data, alpha = 0.8)
ax.set_title('Amount of apartaments for sale and in rent in São Paulo', fontsize = 22)
ax.set_ylabel('Number of apartments', fontsize = 1)
ax.set_xlabel('Status', fontsize = 16)

Second question, how's distribuited the apartaments in the districs  ?

In [ ]:
data['District'].value_counts()


We can see the we have 96 districts in São Paulo, and the top5 are Moema, Mooca, Brás, Bela Vista and Brooklin

Third question, how's our price is distribuited between for sale and rent

In [ ]:
ax = sns.catplot(x = 'Negotiation Type', y = 'Price', kind = 'boxen',
                data = data)


Really weird this data, let's plot in a scatter plot some of these values

In [ ]:
sns.scatterplot(x = 'Condo', y = 'Price', hue = 'Negotiation Type', data = data)


The values for rent are with 0 prices, because they pay for Condo, instead of actually paying to buy the appartment

There is difference of price with apparamets with suites or no ?

In [ ]:
# Let's keep exploring out data
plt.figure(figsize = (12,8))
ax = sns.catplot(x = 'Suites', y = 'Price',data = data[data['Negotiation Type'] == 'sale'], kind = 'boxen');


Yes, for sure. As increase the number of suites in the appartment, grows it's price

Let's see how is related our variable size in our dataset

In [ ]:
plt.figure(figsize = (10,8))
ax = sns.regplot(x = 'Size', y = 'Price', data = data[data['Negotiation Type'] == 'sale'])


We can visualize a positive relation between size and price. Let's visualize the condo for the rent appartments

In [ ]:
plt.figure(figsize = (10,6))
ax = sns.scatterplot(x = 'Size', y = 'Condo', hue = 'Negotiation Type', data = data)

ax.set_title('Condo x Size', fontsize = 22)
ax.set_ylabel('Condo', fontsize = 16)
ax.set_xlabel('Size', fontsize = 16)


If we want we can plot differently

In [ ]:
sns.relplot(x = 'Size', y = 'Condo', data = data, hue = 'Negotiation Type', col = 'Negotiation Type')

## Splitting into rent and sale 
I think the best situation we have is try an analysis differently for each type of negociation¶

In [ ]:
data_rent = data[data['Negotiation Type'] == 'rent']
data_sale = data[data['Negotiation Type'] == 'sale']

*Let's start the analysis for appartaments for sale*

Number of rooms inscrease the final price ?


In [ ]:
# Let's keep exploring out data
ax = sns.catplot(x = 'Rooms', y = 'Price',data = data_sale, kind = 'boxen',
            palette = 'cubehelix')

*We see that the price really increases when the appartment has more than 3 rooms apparently. It's is not really good these analysis, because 4 rooms has a huge outlier and probably the y scale is not really realistic But for what we can see, now, we can assume that*

And for toilets ? Maybe more than 2 toilets increases a lot ?

In [ ]:
plt.figure(figsize =(10,6))
ax = sns.boxplot(x = 'Toilets', y = 'Price', data = data_sale,
                palette = 'plasma')

ax.set_title('Distribution of toilets', fontsize = 22)
ax.set_ylabel('Price', fontsize = 16)
ax.set_xlabel('Size', fontsize = 16)



Really there are appartaments with more than 5 toilets ? xD

Whats the average price for sale in each district ?

In [ ]:
grouped = data_sale.groupby('District')['Price'].mean().reset_index()
grouped = grouped.sort_values(by = 'Price',ascending = False)
plt.figure(figsize = (40,40))
sns.set(font_scale = 2.3)
ax = sns.barplot(x='Price', y='District', data= grouped,palette = 'plasma')

In [ ]:
sns.set(font_scale = 1)

## Now lets visualize the data in rent

In [ ]:
data_rent.describe()

*Maybe the rent dataset is better than data_sale, we will keep our analysis with data_rent*

In [ ]:
# Let's keep exploring out data
plt.figure(figsize =(10,8))
ax = sns.boxplot(x = 'Rooms', y = 'Price',data = data_rent,
            palette = 'cubehelix')


The number of Suites inscreases the price ?

In [ ]:
# Let's keep exploring out data
ax = sns.catplot(x = 'Suites', y = 'Price',data = data_rent, kind = 'boxen',
            palette = 'cubehelix')

The prices increase as the number of suites gets higher...

Which district has higher mean price in São Paulo ?

In [ ]:
grouped = data_rent.groupby('District')['Price'].mean().reset_index()
grouped = grouped.sort_values(by = 'Price',ascending = False)
plt.figure(figsize = (40,40))
sns.set(font_scale = 2.3)
ax = sns.barplot(x='Price', y='District', data= grouped,
                palette = 'plasma')
ax.set_title('Mean Price per District', fontsize = 45)

Itaim bibi, Iguatemi, Pinheiros and Moema they are the top4 in price rent in são paulo

In [ ]:
sns.set(font_scale = 1)

Does number of parkings increases the price ?

In [ ]:
sns.catplot(x='Parking', y='Price', data=data_rent);



I guess furnished appartaments for sure has higher values, lets check this

In [ ]:
plt.figure(figsize =(10,8))
ax = sns.boxplot(x = 'Furnished', y = 'Price',data = data_rent,
            palette = 'cubehelix')


We can't see the difference due the outliers from unfurnished appartaments, but is a slighty diference between the medians

# From here i will digging into our x , y variables



Checking the distribuition of price

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.distplot(data_rent['Price'])


The distribuition for our y is not a normal distribution

In [ ]:
ax = sns.pairplot(data_rent, y_vars = 'Price', x_vars = ['Condo', 'Size', 'Rooms','Toilets','Suites','Parking','Elevator'], height = 5, kind = 'reg')
ax.fig.suptitle('Dispersão entre as Variáveis', fontsize=20, y=1.05)
ax

In [ ]:
data_rent['log_Price'] = np.log(data_rent['Price'])
data_rent['log_Condo'] = np.log(data_rent['Condo'])
data_rent['log_Size'] = np.log(data_rent['Size'])
data_rent['log_Furnished'] = np.log(data_rent['Furnished']+1)
data_rent['log_Swim'] = np.log(data_rent['Swimming Pool']+1)
data_rent['log_Elevator'] = np.log(data_rent['Elevator']+1)

In [ ]:
data_rent.head()

In [ ]:
# correlation plot
plt.figure(figsize = (10,8))
corr = data_rent.corr()
sns.heatmap(corr, cmap = 'plasma', annot= True);

## Declaring x and y

In [ ]:
y = data_rent['log_Price']

In [ ]:
X = data_rent[['log_Size', 'log_Swim', 'log_Furnished']]

In [ ]:
## Train test split
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

# Implementing linear regression

In [ ]:
import statsmodels.api as sm

In [ ]:
X_train_const = sm.add_constant(X_train)
X_train

In [ ]:
X_train_const

In [ ]:
model_statsmodels = sm.OLS(y_train,X_train_const, hasconst = True).fit()

In [ ]:
model_statsmodels.summary()

In [ ]:
print(model_statsmodels.summary())

*Not the best R-Squared but ok.*

## Checking our model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
model = LinearRegression()
model.fit(X_train,y_train)

## Getting our R²

In [ ]:
model.score(X_train,y_train).round(3)

## Getting predictions

In [ ]:
y_pred = model.predict(X_test)

## Getting R² from our y_pred

In [ ]:
metrics.r2_score(y_test, y_pred).round(3)

## Interpreting our coeficient

In [ ]:
print(model.coef_)
print(model.intercept_)

In [ ]:
X_train.columns

In [ ]:
index = ['Intercept','log_Size', 'log_Swim', 'log_Furnished']

Creating dataframe to show the coefs

In [ ]:
pd.DataFrame(data= np.append(model.intercept_, model.coef_), index = index, columns = ['Parameters'])

*Intercept - Excluding all effects from the variables log_size, log_swim and log_furnished, we the mean price of R$ 30*

*log_size = For every 1 % incresead in size we have an increase of 0.94% of the price*


*log_swim = For every 1 % incresead in size we have an increase of 0.36% of the price*


*log_Furnished = For every 1 % incresead in size we have an increase of 0.60% of the price*

## Making predictions with our X_train

In [ ]:
y_pred_train = model.predict(X_train)

In [ ]:
ax = sns.scatterplot(x = y_pred_train, y = y_train)
ax.figure.set_size_inches(12, 6)
ax.set_title('Pred X Real', fontsize=18)
ax.set_xlabel('log Price - Pred', fontsize=14)
ax.set_ylabel('log Price - Real', fontsize=14)
ax

In [ ]:
## Getting residuals
resid = y_train - y_pred_train

In [ ]:
## Ploting the histogram of the residuals
plt.figure(figsize=(10,8))
sns.distplot(resid)
ax.figure.set_size_inches(20, 10)
ax.set_title('Histogram from our resid', fontsize=18)
ax.set_xlabel('log Price', fontsize=14)
ax